In [1]:
!pip install --upgrade openai -q

## OpenAI Function Calling is a powerful feature that allows you to structure the output from language models like GPT-3.5 Turbo and GPT-4 in a way that’s more suitable for machine consumption. Instead of receiving unstructured natural language, you get a structured JSON object that you can use directly in your code. Let’s dive into some practical examples to illustrate how it works:

### Imagine you want to build a chatbot that answers questions by calling external APIs. For instance, let’s create a chatbot that sends an email based on user input.The response from the model will be a JSON document containing the necessary arguments for sending an email which can help us to call python function which will send an email.


In [2]:
import requests
import json
import openai
import os

In [3]:
os.environ['gpt_key'] = "..."

In [4]:
# Set your OpenAI API key here
openai.api_key = os.environ.get('gpt_key')

In [5]:
function = {
    "name": "send_email",
    "description": "template to have an email sent.",
    "parameters": {
        "type": "object",
        "properties": {
            "to_address": {"type": "string", "description": "To address for email"},
            "body": {"type": "string", "description": "Body of the email"},
            "date": {"type": "string", "description": "the date the email must be sent."},
            "time": {"type": "string", "description": "the time the email must be sent."}
        }
    }
}

In [6]:
# Define the chat message and function
chat_message = [
    {"role": "user", "content": "Send Cobus from HumanFirst AI an email and ask him for the sales forecast spreadsheet. Schedule the mail for tomorrow at 12 noon."}
]



# Make the API call
response = requests.post(
    "https://api.openai.com/v1/chat/completions",
    json={
        "model": "gpt-4-0613",
        "messages": chat_message,
        "functions": [function]
    },
    headers={
        "Content-Type": "application/json",
        "Authorization": f"Bearer {openai.api_key}"
    }
)

In [7]:
email_details = response.json()
email_details

{'id': 'chatcmpl-8yKiEiHlqbVTYhcDtV9jWBoOeiIqh',
 'object': 'chat.completion',
 'created': 1709389990,
 'model': 'gpt-4-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': None,
    'function_call': {'name': 'send_email',
     'arguments': '{\n  "to_address": "cobus@humanfirst.ai",\n  "body": "Dear Cobus, \\n\\nI hope this email finds you well. Could you please share the sales forecast spreadsheet with me? \\n\\nBest regards,",\n  "date": "tomorrow",\n  "time": "12:00"\n}'}},
   'logprobs': None,
   'finish_reason': 'function_call'}],
 'usage': {'prompt_tokens': 117, 'completion_tokens': 76, 'total_tokens': 193},
 'system_fingerprint': None}

In [8]:
email_details = json.loads(email_details["choices"][0]['message']['function_call']['arguments'])

In [9]:
# Now you can use the email_details to send the actual email
# Implement your email sending logic here

print("Email details:")
print(f"To: {email_details['to_address']}")
print(f"Body: {email_details['body']}")
print(f"Date: {email_details['date']}")
print(f"Time: {email_details['time']}")

Email details:
To: cobus@humanfirst.ai
Body: Dear Cobus, 

I hope this email finds you well. Could you please share the sales forecast spreadsheet with me? 

Best regards,
Date: tomorrow
Time: 12:00
